<a href="https://colab.research.google.com/github/skyfly200/battery-holder/blob/main/battery_holder_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
if 'google.colab' in str(get_ipython()):
  !pip install git+https://github.com/FullControlXYZ/fullcontrol --quiet
import fullcontrol as fc
from google.colab import files
from math import tau
import numpy as np

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [174]:
# printer/gcode parameters

design_name = 'battery_holder'
nozzle_temp = 210
bed_temp = 60
print_speed = 8000
fan_percent = 100
printer_name='prusa_i3' # generic / ultimaker2plus / prusa_i3 / ender_3 / cr_10 / bambulab_x1 / toolchanger_T0

In [195]:
# design parameters
clearance = 0.5
battery_diameter = clearance + 18#mm
battery_radius = battery_diameter / 2
holder_depth = 8#mm
cells_x = 4
cells_y = 4
facets = 16

EW = 0.6
# Extrusion Width (mm) - Width of printed lines - recommended value: 1.5x nozzle diameter
# default value: 0.6

EH = 0.3
# Extrusion Height (mm) - Height of printed lines (i.e. layer thickness) - recommended value: 0.5x nozzle diameter
# default value: 0.2

In [196]:
# generate the design (make sure you've run the above cells before running this cell)

layers = int(holder_depth/EH)

centre = fc.Point(x=100,y=100,z=0)

steps = []

for i in range(cells_x+1):
  for j in range(cells_y+1):
    x_edge = (i == 0 or i == cells_x)
    y_edge = (j == 0 or j == cells_y)
    x = (i) * (battery_radius + EW) * 2  + centre.x
    y = (j) * (battery_radius + EW) * 2  + centre.y
    s = 5 * EW
    px = s * 0.5 if i == 0 else -s * 1.5
    py = s * 0.5 if j == 0 else -s * 1.5
    s2 = 2.5 * EW
    px2 = s * 0.75 if i == 0 else -s * 1.25
    py2 = s * 0.75 if j == 0 else -s * 1.25

    if x_edge and y_edge:
      # corner piece - just one square
      steps.append(fc.PrinterCommand(id='retract'))
      steps.extend(fc.travel_to(fc.Point(x=x+px, y=y+py, z=0)))
      steps.append(fc.PrinterCommand(id='unretract'))
      steps.extend(fc.rectangleXY(fc.Point(x=x+px, y=y+py, z=0), s, s, False))
      steps.extend(fc.rectangleXY(fc.Point(x=x+px2, y=y+py2, z=0), s2, s2, False))
    elif y_edge:
      r = 1.5 if j == 0 else -5
      # edge - two inset rectangles
      steps.append(fc.PrinterCommand(id='retract'))
      steps.extend(fc.travel_to(fc.Point(x=x+px+EW*1.5, y=y+EW*2.25+r, z=0)))
      steps.append(fc.PrinterCommand(id='unretract'))
      steps.extend(fc.rectangleXY(fc.Point(x=x+px+EW*1.5, y=y+EW*2.25+r, z=0), 12*EW, 1.5*EW, False))
      steps.extend(fc.rectangleXY(fc.Point(x=x+px, y=y+EW+r, z=0), 15*EW, 4*EW, False))
    elif x_edge:
      r = 1.5 if i == 0 else -5
      # edge - two inset rectangles
      steps.append(fc.PrinterCommand(id='retract'))
      steps.extend(fc.travel_to(fc.Point(x=x+EW*2.25+r, y=y+py+EW*1.5, z=0)))
      steps.append(fc.PrinterCommand(id='unretract'))
      steps.extend(fc.rectangleXY(fc.Point(x=x+EW*2.25+r, y=y+py+EW*1.5, z=0), 1.5*EW, 12*EW, False))
      steps.extend(fc.rectangleXY(fc.Point(x=x+EW+r, y=y+py, z=0), 4*EW, 15*EW, False))
    else:
      # inside rectangle - three inset squares
      loops = 7
      spacing = 1.1
      for k in range(loops):
        o = 1 + spacing * k
        if(k == 0): steps.append(fc.PrinterCommand(id='retract'))
        steps.extend(fc.travel_to(fc.Point(x=x-EW*o, y=y-EW*o, z=0)))
        if(k == 0): steps.append(fc.PrinterCommand(id='unretract'))
        steps.append(fc.Point(x=x+EW*o, y=y-EW*o, z=0))
        steps.append(fc.Point(x=x+EW*o, y=y+EW*o, z=0))
        steps.append(fc.Point(x=x-EW*o, y=y+EW*o, z=0))
        steps.append(fc.Point(x=x-EW*o, y=y-EW*o, z=0))

# TODO - turn on fan and speed up feef rate here after 1st layer

steps_cir = []

for i in range(facets+1):
  angle =  tau * (i / facets + 0.25)
  x = (battery_radius + EW*0.75) * np.cos(angle) + centre.x + battery_radius + EW
  y = (battery_radius + EW*0.75) * np.sin(angle) + centre.y + battery_radius + EW
  if i == 0:
    steps_cir.append(fc.PrinterCommand(id='retract'))
    steps_cir.extend(fc.travel_to(fc.Point(x=x, y=y, z=0)))
    steps_cir.append(fc.PrinterCommand(id='unretract'))
  else:
    steps_cir.append(fc.Point(x=x, y=y, z=0))

steps_cir = fc.move(steps_cir,fc.Vector(x=battery_diameter + 2 * EW),copy=True, copy_quantity=cells_x)
steps_cir = fc.move(steps_cir,fc.Vector(y=battery_diameter + 2 * EW),copy=True, copy_quantity=cells_y)
steps_cir = fc.move(steps_cir,fc.Vector(z=EH),copy=True, copy_quantity=layers)

steps.extend(steps_cir)

In [198]:
# preview the design

#fc.transform(steps, 'plot', fc.PlotControls(style='line'))
# hover the cursor over the lines in the plot to check xyz positions of the points in the design

# uncomment the next line to create a plot with real heights/widths for extruded lines to preview the real 3D printed geometry
fc.transform(steps, 'plot', fc.PlotControls(zoom=0.7, style='tube', initialization_data={'extrusion_width': EW, 'extrusion_height': EH}))

# uncomment the next line to create a neat preview (click the top-left button in the plot for a .png file) - post and tag @FullControlXYZ :)
# fc.transform(steps, 'plot', fc.PlotControls(neat_for_publishing=True, zoom=0.7,  initialization_data={'extrusion_width': EW, 'extrusion_height': EH}))


In [188]:
# generate and save gcode

gcode_controls = fc.GcodeControls(
    printer_name=printer_name,

    initialization_data={
        'primer': 'travel',
        'print_speed': print_speed,
        'nozzle_temp': nozzle_temp,
        'bed_temp': bed_temp,
        'fan_percent': fan_percent,
        'extrusion_width': EW,
        'extrusion_height': EH})
gcode = fc.transform(steps, 'gcode', gcode_controls)
open(f'{design_name}.gcode', 'w').write(gcode)
files.download(f'{design_name}.gcode')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#### please tell us what you're doing with FullControl!

- tag FullControlXYZ on social media ([twitter](https://twitter.com/FullControlXYZ), [instagram](https://www.instagram.com/fullcontrolxyz/), [linkedin](https://www.linkedin.com/in/andrew-gleadall-068587119/), [tiktok](https://www.tiktok.com/@fullcontrolxyz))
- email [info@fullcontrol.xyz](mailto:info@fullcontrol.xyz)
- post on the [subreddit](https://reddit.com/r/fullcontrol)
- post in the [github discussions or issues tabs](https://github.com/FullControlXYZ/fullcontrol/issues)

in publications, please cite the original FullControl paper and the github repo for the new python version:

- Gleadall, A. (2021). FullControl GCode Designer: open-source software for unconstrained design in additive manufacturing. Additive Manufacturing, 46, 102109.
- Gleadall, A. and Leas, D. (2023). FullControl [electronic resource: python source code]. available at: https://github.com/FullControlXYZ/fullcontrol